In [56]:
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import torch
import time

In [45]:
def get_dataset():
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        training_data = datasets.CIFAR10(download=True, root='Implementation\data', train=True, transform=transform)
        testing_data = datasets.CIFAR10(download=True, root='Implementation\data', train=False, transform=transform)
        return training_data, testing_data

In [46]:
def get_dataloader(data):
    return DataLoader(dataset=data, batch_size=32, shuffle=True, num_workers=3, pin_memory=True)


In [47]:
training_data, testing_data = get_dataset()

Files already downloaded and verified
Files already downloaded and verified


In [48]:
validation_data = random_split(training_data, [int((1 - 0.2) * len(training_data)), int(0.2 * len(training_data))])

In [49]:
train_dl, val_dl, test_dl = get_dataloader(training_data), get_dataloader(validation_data), get_dataloader(testing_data)

In [50]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [51]:
vgg11 = models.vgg11(pretrained=True)
vgg11.to(device=device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [52]:
optimizer = torch.optim.SGD(params=vgg11.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [55]:
print(len(train_dl))

1563


In [57]:
for i in range(20):
    curr_train_loss = 0.
    avg_train_loss = 0.
    curr_val_loss = 0.
    avg_val_loss = 0.
    correct = 0
    total = 0
    for i, data in enumerate(train_dl):
        start = time.time()
        inputs, labels = data
        optimizer.zero_grad()
        outputs = vgg11(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        curr_train_loss += loss.item()
        end = time.time()
        print(f'Time for one iteration: {end - start}')
    for i, data in enumerate(val_dl):
        inputs, labels = data
        outputs = vgg11(inputs)
        vloss = loss(outputs, labels)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        curr_val_loss += vloss.item()

    avg_train_loss = curr_train_loss / len(train_dl)
    avg_val_loss = curr_val_loss / len(val_dl)
    print(f'Epoch: {i}\n  Avg Training Loss: {round(avg_train_loss, 3)}\n  Avg Validation Loss: {round(avg_val_loss, 3)}\n  Validation Accuracy: {(correct / total): .2%}')

Time for one iteration: 2.967925786972046
Time for one iteration: 1.3779778480529785
Time for one iteration: 1.335059642791748
Time for one iteration: 1.0571751594543457
Time for one iteration: 1.085378646850586
Time for one iteration: 1.0487284660339355
Time for one iteration: 1.1275794506072998
Time for one iteration: 1.0009105205535889
Time for one iteration: 1.0388751029968262
Time for one iteration: 0.9856233596801758
Time for one iteration: 1.0131065845489502
Time for one iteration: 0.9849426746368408
Time for one iteration: 0.9975295066833496
Time for one iteration: 1.0640063285827637
Time for one iteration: 1.0002501010894775
Time for one iteration: 1.6979026794433594
Time for one iteration: 2.796537160873413
Time for one iteration: 1.893852710723877
Time for one iteration: 1.2198412418365479
Time for one iteration: 1.3508505821228027
Time for one iteration: 1.1574687957763672
Time for one iteration: 1.3571577072143555
Time for one iteration: 1.3892028331756592
Time for one ite

KeyboardInterrupt: 